## Algorithm for ITCZ Identification in Tropical Atlantic Ocean
*Algoritmo para Identificação da ZCIT no Oceano Atlântico Tropical*

> ##### **Version 01 - December 4th 2023**
*Versão 01 - 4 de Dezembro de 2023*

> #### **Authors**: *Natan Nogueira, André Reis, Pedro Machado e Michelle Reboita*
*Autores: Natan Nogueira, André Reis, Pedro Machado e Michelle Reboita*

> #### **Input**: *Global Precipitation Climatatology Project (GPCP) Climate Data Record (CDR), Daily V1.3*

> #### **Output**: *ITCZ Position, Width and Intensity *
*Posição, Largura e Intensidade da ZCIT*

#### Considerations
*Considerações*

1.   For this article, it was used daily precipitation (mm) from 1997 to 2022, but it can be used with any type of precipitation data depending on your purpose.

 *Para esse artigo, é utilizado precipitação diária (mm) de 1997 a 2022, mas pode ser usado qualquer tipo de dados de precipitação dependendo do seu propósito.*

2.   Also, there is no specific area or time to use this algorithm. In this case, it was delimited to Tropical Atlantic Ocean (18°N to 8°S and 49°W to 10°W) in pentads (mean of 5 days). Then, if necessary, those parameters can be changed.

 *Também, não há uma área e tempo específico para usar esse algoritmo. Nesse caso, foi delimitado para o Oceano Atlântico Tropical (18°N a 8°S and 49°O a 10°O) em pêntadas (média de 5 dias). Logo, se necessário, esses parâmetros podem ser mudados.*

3. The threshold used in this study was the absolute mean data (mean in all grid points and times) to filter all grid points above the threshold in order to identify which ones belong to the ITCZ band in booleans (0 - indicates precipitation below the threshold and 1 - indicates precipitation above the threshold), so if necessary it can be changed.

 *O limiar usado nesse estudo foi a média absoluta dos dados (média em todos os pontos de grade e tempos) para filtrar todos os pontos de grade acima do limiar a fim de identificar quais pertecem à banda da ZCIT em booleanos (0 - indica precipitação abaixo do limiar e 1 - indica precipitação acima do limiar), então se necessário pode ser modificado.*

4.  The grid points from input data have 1° of spatial resolution and in this study the ITCZ band was identified when there was at least 2° (2 grid points) from top-down (from the northernmost latitude to southernmost latitude) in each longitude from left to right (from the westernmost longitude to easternmost longitude).

 *Os pontos de grade dos dados input tem 1° de resolução espacial e nesse estudo a banda da ZCIT é identificada quando há pelo menos 2° (2 pontos de grade) de cima para baixo (da latitude mais ao norte para a latitude mais ao sul) em cada longitude da esquerda para a direita (da longitude mais a oeste para a longitude mais a leste).*

5. Therefore, if you want a different ITCZ analysis, you must change the value (variable used to sum the booleans from each grid point). In this case, the value used to identify the ITCZ band is above 1.

 *Portanto, se você quer uma análise diferente da ZCIT, você precisa alterar o value (variável que soma os booleanos de cada ponto). Nesse caso, o value usado para identificação da banda da ZCIT é acima de 1.*

In [2]:
#Libraries Installation / Instalação das bibliotecas
!pip install metpy
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libeccodes-dev libproj-dev
!pip install importlib-metadata
!apt-get install libgeos-dev
!pip install cartopy
!pip uninstall -y shapely
!pip install --no-binary shapely shapely --force
!pip install geopandas
!pip install pygrib
!pip install netCDF4
!pip install datetime
!pip install scipy
!pip install seasborn
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.5/400.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 7.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libproj-dev is already the newest version (8.2.1-1).
libproj-dev set to manually installed.
proj-data is already the newest version (8.2.1-1).
proj-data set to manually installed.
The following NEW packages will be installed:
  proj-bin
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 154 kB of archives.
After this operation, 492 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 proj-bin amd64 8.2.1-1 [154 kB]
Fetched 154 kB in 1s (167 kB/s)
Selecting previously unselected package proj-bin.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../proj-bin_8.2.1-1_amd64.deb ...
Unpacking proj-bin (8.2.1-1) ...
Set

In [3]:
#Uploading the libraries / Carregando as bibliotecas
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.shapereader as shpreader
import geopandas
import pandas as pd
import datetime


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Adding the input and output path / Adicionando os caminhos de entrada e saída
pathin = '/content/drive/MyDrive/IC/'
pathout = '/content/drive/MyDrive/IC/figs'

In [8]:
#Combining and Concatenating the files .nc / Combinando e concatenando os arquivos .nc
ds2 = xr.open_mfdataset(pathin + '/Cópia de precip1.nc', combine='nested', concat_dim='time', parallel=True)

#Transforming the longitudes in -180° to 180° format / Transformando as longitudes em formato de -180° a 180°
#Latitudes values are in the correct order / Os valores das latitudes estão na ordem correta
ds2.coords['longitude'] = ((ds2.coords['longitude'] + 180) % 360) - 180

#Organizing the longitudes / Organizando as longitudes
ds2 = ds2.sortby(ds2.longitude)

#Slicing the longitudes to the study area / Cortando as longitude para a área de estudo
ds = ds2.sel(longitude=slice(-49, -18),latitude=slice(-8, 18))
ds

/usr/local/lib/python3.10/dist-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'gini' loading failed:
module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:         (time: 1461, bnds: 2, longitude: 32, latitude: 27)
Coordinates:
  * time            (time) datetime64[ns] 2009-01-01 2009-01-02 ... 2012-12-31
  * longitude       (longitude) float32 -49.0 -48.0 -47.0 ... -20.0 -19.0 -18.0
  * latitude        (latitude) float32 -8.0 -7.0 -6.0 -5.0 ... 16.0 17.0 18.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds       (time, bnds) datetime64[ns] dask.array<chunksize=(1461, 2), meta=np.ndarray>
    longitude_bnds  (time, longitude, bnds) float32 dask.array<chunksize=(1461, 32, 2), meta=np.ndarray>
    latitude_bnds   (time, latitude, bnds) float32 dask.array<chunksize=(1461, 27, 2), meta=np.ndarray>
    precip          (time, latitude, longitude) float32 dask.array<chunksize=(1461, 27, 32), meta=np.ndarray>
Attributes: (12/47)
    CDI:                        Climate Data Interface version 2.0.4 (https:/...
    source:                     /data1/GPCP_CDR/GPCP_Output/1DD//bin/200901/s...
    institution:                ACADEMIC > UMD/ESSIC > Earth System Science I...
    Conventions:                CF-1.6, ACDD 1.3
    title:                      Global Precipitation Climatatology Project (G...
    references:                 Huffman et al. 1997, http://dx.doi.org/10.117...
    ...                         ...
    product_version:            v01r03
    platform:                   GOES (Geostationary Operational Environmental...
    sensor:                     Imager, AIRS > Atmospheric Infrared Sounder, ...
    spatial_resolution:         1 degree
    comment:                    Processing computer: eagle2.umd.edu
    CDO:                        Climate Data Operators version 2.0.4 (https:/...

In [9]:
#Organizing the time / Organizando os tempos
ds = ds.sortby(ds.time)

#Calculating the mean precipitation in pentads (mm/day) / Calculando a média da precipitação em pêntadas (mm/dia)
pentad_ds = ds.resample(time='5D', closed='right', label='right').mean()
pentad_ds

<xarray.Dataset>
Dimensions:         (longitude: 32, time: 293, bnds: 2, latitude: 27)
Coordinates:
  * longitude       (longitude) float32 -49.0 -48.0 -47.0 ... -20.0 -19.0 -18.0
  * latitude        (latitude) float32 -8.0 -7.0 -6.0 -5.0 ... 16.0 17.0 18.0
  * time            (time) datetime64[ns] 2009-01-01 2009-01-06 ... 2012-12-31
Dimensions without coordinates: bnds
Data variables:
    longitude_bnds  (time, longitude, bnds) float32 dask.array<chunksize=(1, 32, 2), meta=np.ndarray>
    latitude_bnds   (time, latitude, bnds) float32 dask.array<chunksize=(1, 27, 2), meta=np.ndarray>
    precip          (time, latitude, longitude) float32 dask.array<chunksize=(1, 27, 32), meta=np.ndarray>
Attributes: (12/47)
    CDI:                        Climate Data Interface version 2.0.4 (https:/...
    source:                     /data1/GPCP_CDR/GPCP_Output/1DD//bin/200901/s...
    institution:                ACADEMIC > UMD/ESSIC > Earth System Science I...
    Conventions:                CF-1.6, ACDD 1.3
    title:                      Global Precipitation Climatatology Project (G...
    references:                 Huffman et al. 1997, http://dx.doi.org/10.117...
    ...                         ...
    product_version:            v01r03
    platform:                   GOES (Geostationary Operational Environmental...
    sensor:                     Imager, AIRS > Atmospheric Infrared Sounder, ...
    spatial_resolution:         1 degree
    comment:                    Processing computer: eagle2.umd.edu
    CDO:                        Climate Data Operators version 2.0.4 (https:/...

In [10]:
#Calculating the threshold (absolute data mean) / Calculando o limiar (média absoluta dos dados)
limiar=ds.precip.mean(dim=('longitude','latitude',"time"))
limiar.values

array(3.3624818, dtype=float32)

In [11]:
#Filtering the grid points above the threshold / Filtrando os pontos de grade acima do limiar
precip_maior = pentad_ds.precip > limiar.values
precip_maior

<xarray.DataArray 'precip' (time: 293, latitude: 27, longitude: 32)>
dask.array<gt, shape=(293, 27, 32), dtype=bool, chunksize=(1, 27, 32), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 -49.0 -48.0 -47.0 -46.0 ... -20.0 -19.0 -18.0
  * latitude   (latitude) float32 -8.0 -7.0 -6.0 -5.0 ... 15.0 16.0 17.0 18.0
  * time       (time) datetime64[ns] 2009-01-01 2009-01-06 ... 2012-12-31

# Locating ITCZ



In [12]:
#Defining the array's shape with the same size from xarray's longitude and time
#with precipitation data above the threshold (boolean: 0 - indicates precipitation
# below the threshold and 1 - indicates precipitation above the threshold) until 3 possibles ITCZ bands
#/ Definindo o tamanho das array com mesmo tamanho das dimensões longitude
# e tempo do xarray com os dados de precipitação acima do limiar (booleano:
# 0 para abaixo do limiar e 1 para acima do limiar) para até 3 'possíveis' bandas da zcit
shape=(len(precip_maior.longitude), len(precip_maior.time), 3)

#Creating two empty arrays to collect the initial and final latitude ICTZ band's data
#with the same previous shape / Criando duas arrays vazias para coletar os dados da
# latitude inicial e final da banda da zcit com o mesmo shape anterior
arrayI=np.full(shape, np.nan)
arrayF=np.full(shape, np.nan)

In [13]:
#Defining the function with input xarray with precipitation above the threshold (V) and
#Latitudes from the study area with the reverse order in order to identify the ICTZ top-down ()
#from the northernmost latitude to southernmost latitude) in each longitude from left to right
#(from the westernmost longitude to easternmost longitude) /
#Definindo a função com a entrada da xarray com os dados de precipitação acima do limiar (V)
# e latitudes da área de estudo com a ordem inversa com intuito do algoritmo identificar
#a ZCIT da latitude mais norte para a mais sul (De cima para baixo) e da longitude mais oeste para leste (da esquerda para direita)

def find_zcit(V,lat):
    #Creating a list to store the initial latitude / Criando a lista para armazenar a latitude inicial
    lati=[]
    #Creating a list to store the final latitude / Criando a lista para armazenar a latitude final
    latf=[]
    #Values' Accumulator 0 or 1 depending if the grid point is below or above the threshold, respectively /
    #Acumulador de valores 0 ou 1 dependo se o ponto de grade é abaixo ou acima do limiar, respectivamente
    value=0
    #Counter that verifies how many grid points has the precipitation aboce the threshold (ITCZ width, minimum 2° - 2 grid points) /
    #Contador que verifica quantos pontos de grade tem precipitação acima do limiar (largura da ZCIT, mínimo de 2° - 2 pontos de grade)
    cont=0
    #Control variable of the ITCZ's band number (zcit=1 - ITCZ, zcit=2 - Double ITCZ)
    #Variável de controle do número de bandas da ZCIT (zcit=1 - ZCIT primária, zcit=2 - ZCIT secundária)
    zcit=0

    #Sweeping all grid points in latitude for each longitude [val] and time [i]
    #Varredura de todos os pontos em latitude para cada longitude [val] e tempo [i]
    for i, val in enumerate(V):
        #Begin of Sweeping / Início da varredura
        cont=cont+1
        #Quantification of val: 0 or 1 / Quantificação do val: 0 ou 1
        value=value+val

        #Initial latitude from ITCZ located / Latitude inicial de uma ZCIT localizada
        if value==1:
            #Register of latitude in the time [i] into li variable / Registro da latitude no tempo [i] na variável li
            li=lat[i]

        #Only on grid point was acknowledged, it's not classified as ICTZ /
        #Apenas um ponto de grade foi reconhecido, não é classificada como ZCIT
        if value != cont and value==1:
            #Resets the Values' Accumulator in order to continue the identification / Zera o acumulador de valores para continuar a identificação
            value=0
            #Resets the counter in order to continue the identification / Zera o contador para continuar a identificação
            cont=0

        #Any value with the precipitation above the threshold was found, Absence of ITCZ /
        #Nenhum valor com precipitação acima do limiar foi encontrado, ausência de ZCIT
        if value != cont and value==0:
            #Resets the Values' Accumulator in order to continue the identification / Zera o acumulador de valores para continuar a identificação
            value=0
            #Resets the counter in order to continue the identification / Zera o contador para continuar a identificação
            cont=0

        #Final Latitude of some ITCZ with at least 2 grid points was found /
        #Latitude final de uma ZCIT com pelo menos 2 pontos de grade foi encontrada
        if value != cont and value>1:
            #Enumeration of the ITCZ (zcit=1), continuation of the process in the same longitude due to a possible Double ITCZ /
            #Enumeração da ZCIT primária (zcit=1), continuação do processo na mesma longitude para possível ZCIT secundária
            zcit=zcit+1
            #Resets the Values' Accumulator in order to continue the identification / Zera o acumulador de valores para continuar a identificação
            value=0
            #Resets the counter in order to continue the identification / Zera o contador para continuar a identificação
            cont=0
            #Stores the ITCZ initial latitude / Armazena-se a latitude inicial da ZCIT
            lati.append(li)
            #Stores the ITCZ final latitude: previous latitude - lat[i-1], because the values' accumulator and counter
            #are different, that is, the current latitude has value=0 and the counter has an extra value (value=cont+1)
            #Armazena-se a latitude final da ZCIT: latitude anterior da atual - lat[i-1], pois o acumulador de valores e contador
            #são diferentes, ou seja, a latitude atual possui value=0 e contador tem um valor a mais (value=cont+1)
            latf.append(lat[i-1])

    return lati,latf  #Return the initial and final latitude of the band, respectively / Retorna-se a latitude inicial e final das bandas, respectivamente


In [14]:
lat=precip_maior.latitude.values[::-1]  #Latitude values reversed / Valores de latitude invertidos
lon=precip_maior.longitude.values       #Longitude values / Valores de longitude
time=precip_maior.time.values           #Time values / Valores do tempo
precip_maior_array=precip_maior.values  #Xarry values with precipitation above the threshold / Valores da xarray com a precipitação acima do limiar

In [15]:
#Running the find_zcit function / Executando a função find_zcit

#Negative time to do the iteration inside the loop to store the initial and final latitude into the arrays /
#Tempo negativo para poder fazer a iteração dentro do for para armazenar nas arrays das latitudes iniciais e finais
t=-1

#For each xarray's time the find_zcit function is done /
#Para os cada tempo da xarray é feito a função find_zcit
for timeI in time:
    #Iteration in time / Iteração do tempo
    t=t+1
    #Initial iteration of the longitude / Iteração inicial da longitude
    l=-1

    #For each longitude in the same time the find_zcit function is done /
    #Para cada longitude no mesmo tempo é feita a função find_zcit
    for lonI in lon:
        #Iteration of the longitude / Iteração da longitude
        l=l+1
        #Nominating the xarray with specific time and longitude with reversed latitudes /
        #Chamando a xarray com o tempo e longitude específico com as latitudes invertidas
        V=precip_maior_array[t,:,l][::-1]
        #Applying the function to the initial and final latitudes /
        #Aplicando a função para as latitudes iniciais e finais
        lati,latf=find_zcit(V,lat)
        #Initial iteration of the zcit / Iteração inicial da zcit
        zcit=-1

        #Looping to run the find_zcit function and to store the latitudes in the correct arrays /
        #Looping para executar a função find_zcit e armazenar as latitudes nas arrays correta
        for lat_1, lat_2 in zip(lati,latf):
            #Defining the dimension of which ITCZ (zcit=1 - first, zcit=1 - second ...) /
            #Definição da dimensão de qual ZCIT (zcit=1 - primária, zcit=1 - secundária)
            zcit=zcit+1
            #Storing the longitude, time and band of the ITCZ for each initial latitude into the array's indexes /
            #Armazenando nos índices de longitude, tempo e banda da ZCIT da array os valores de cada latitude inicial
            arrayI[l,t,zcit]=lat_1
            #Storing the longitude, time and band of the ITCZ for each final latitude into the array's indexes /
            #Armazenando nos índices de longitude, tempo e banda da ZCIT da array os valores de cada latitude final
            arrayF[l,t,zcit]=lat_2

In [16]:
#Identifying the arrays with the initial and final latitude from each ITCZ band
#/Identificando as arrays de latitude inicial e final de cada banda de ZCIT

#ITCZ initial latitude / latitude inicial da ZCIT primária
arrayI1 = arrayI[:,:,0]
#ITCZ final latitude / latitude final da ZCIT primária
arrayF1 = arrayF[:,:,0]
#Double ITCZ initial latitude / latitude inicial da ZCIT secundária
arrayI2 = arrayI[:,:,1]
#Double ITCZ final latitude / latitude final da ZCIT secundária
arrayF2 = arrayF[:,:,1]
#Third ITCZ initial latitude (identification of spurious data)
#/latitude inicial da ZCIT terciária (identificação de dados espúrios)
arrayI3 = arrayI[:,:,2]
#Third ITCZ final latitude (identification of spurious data)
#/latitude final da ZCIT terciária (identificação de dados espúrios)
arrayF3 = arrayF[:,:,2]


In [17]:
#Creating the Dataset with the created arrays / Criando o Dataset com as arrays criadas
data_new = xr.Dataset(
    {
        "zcit_latI_1": (("longitude", "time"), arrayI1),
        "zcit_latF_1": (("longitude", "time"), arrayF1),
        "zcit_latI_2": (("longitude", "time"), arrayI2),
        "zcit_latF_2": (("longitude", "time"), arrayF2),
        "zcit_latI_3": (("longitude", "time"), arrayI3),
        "zcit_latF_3": (("longitude", "time"), arrayF3),
    },
    coords={"longitude": lon, "time": time}
)

print(data_new)



<xarray.Dataset>
Dimensions:      (longitude: 32, time: 293)
Coordinates:
  * longitude    (longitude) float32 -49.0 -48.0 -47.0 ... -20.0 -19.0 -18.0
  * time         (time) datetime64[ns] 2009-01-01 2009-01-06 ... 2012-12-31
Data variables:
    zcit_latI_1  (longitude, time) float64 13.0 7.0 4.0 11.0 ... 5.0 5.0 3.0 5.0
    zcit_latF_1  (longitude, time) float64 8.0 1.0 -7.0 9.0 ... 2.0 2.0 2.0 3.0
    zcit_latI_2  (longitude, time) float64 4.0 nan nan -5.0 ... nan nan nan nan
    zcit_latF_2  (longitude, time) float64 -7.0 nan nan -6.0 ... nan nan nan nan
    zcit_latI_3  (longitude, time) float64 nan nan nan nan ... nan nan nan nan
    zcit_latF_3  (longitude, time) float64 nan nan nan nan ... nan nan nan nan


In [18]:
#Identifying the grid points with precipitation above the threshold / Identificando os pontos de grade com precipitação acima do limiar
pr = pentad_ds.where(pentad_ds.precip > limiar.values)
pr.precip

<xarray.DataArray 'precip' (time: 293, latitude: 27, longitude: 32)>
dask.array<where, shape=(293, 27, 32), dtype=float32, chunksize=(1, 27, 32), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 -49.0 -48.0 -47.0 -46.0 ... -20.0 -19.0 -18.0
  * latitude   (latitude) float32 -8.0 -7.0 -6.0 -5.0 ... 15.0 16.0 17.0 18.0
  * time       (time) datetime64[ns] 2009-01-01 2009-01-06 ... 2012-12-31
Attributes:
    standard_name:  lwe_precipitation_rate
    long_name:      NOAA Climate Data Record (CDR) of Daily GPCP Satellite-Ga...
    units:          mm/day
    cell_methods:   area: mean time: mean

In [19]:
#Creating a copy of the data with grid points with precipitation above the threshold to filter grid points (only 1°) that are not included in the analyses /
#Criando uma cópia dos dados com precipitação acima do limiar para filtrar pontos de grade (apenas 1°) que não estão incluídos nas análises
zcit_precip_filtered = pr.precip.copy()

#Creating a boolean filter for each longitude, time, initial and final latitude combination /
# Criando uma máscara booleana para cada combinação de longitude, tempo, latitude inicial e latitude final
mask_latitudes = ((pr.precip['latitude'] <= data_new['zcit_latI_1']) & (pr.precip['latitude'] >= data_new['zcit_latF_1'])) | ((pr.precip['latitude'] <= data_new['zcit_latI_2']) & (pr.precip['latitude'] >= data_new['zcit_latF_2']))

#Applying the filter to select the wanted data /
# Aplique a máscara para selecionar os dados desejados
zcit_precip_filtered = zcit_precip_filtered.where(mask_latitudes, np.nan)